In [5]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.datasets import MNIST

In [2]:
input_size = 784
hidden_size = 400
out_size = 10
epochs = 10
batch_size = 100
learning_rate = 0.001

In [11]:
train_data = MNIST(root = './data', train=True, download=True, transform=transforms.ToTensor())
test_data = MNIST(root = './data', train=False, download=True, transform=transforms.ToTensor())

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!


In [17]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = False)


In [20]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, out_size)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
        

In [21]:
net = Net(input_size, hidden_size, out_size)
CUDA = torch.cuda.is_available()
if CUDA:
    net = net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

In [34]:
correct_train = 0
total_train = 0
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
        optimizer.zero_grad()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total_train +=labels.size(0)
        if CUDA:
            correct_train+= (predicted.cpu() == labels.cpu()).sum()
        else:
            correct_train+= (predicted == labels).sum()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1)%100 ==0:
            print(f"Epoch {epoch}/{epochs}, Iteration {i}/{len(train_data)//batch_size}, Loss: {loss.data}, Accuracy: {float(correct_train)/float(total_train)}")

Epoch 0/10, Iteration 99/600, Loss: 0.09102413058280945, Accuracy: 0.9997
Epoch 0/10, Iteration 199/600, Loss: 8.306503332278226e-06, Accuracy: 0.99965
Epoch 0/10, Iteration 299/600, Loss: 0.00046574592124670744, Accuracy: 0.9996
Epoch 0/10, Iteration 399/600, Loss: 0.0003614187298808247, Accuracy: 0.99965
Epoch 0/10, Iteration 499/600, Loss: 0.0007773446850478649, Accuracy: 0.99954
Epoch 0/10, Iteration 599/600, Loss: 2.9277800877025584e-06, Accuracy: 0.9995666666666667
Epoch 1/10, Iteration 99/600, Loss: 4.974365219823085e-05, Accuracy: 0.9995857142857143
Epoch 1/10, Iteration 199/600, Loss: 0.0046136523596942425, Accuracy: 0.9995875
Epoch 1/10, Iteration 299/600, Loss: 6.0558319091796875e-05, Accuracy: 0.9994888888888889
Epoch 1/10, Iteration 399/600, Loss: 0.023129180073738098, Accuracy: 0.9994
Epoch 1/10, Iteration 499/600, Loss: 0.0023356007877737284, Accuracy: 0.9991727272727273
Epoch 1/10, Iteration 599/600, Loss: 3.947257937397808e-05, Accuracy: 0.9989083333333333
Epoch 2/10, 

In [36]:
labels.size(0)

100

In [37]:
# Testing
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    if CUDA:
        images = images.cuda()
        
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total+=labels.size(0)
    
    if CUDA:
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
        correct += (predicted == labels).sum()

print(f"Accuracy: {float(correct)/ float(total)}")

Accuracy: 0.9832
